In [1]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain import hub

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_openai import ChatOpenAI
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
openai = ChatOpenAI(model='gpt-4o-mini', temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")

/home/user1/miniconda3/envs/openai/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
# naver news
loader = WebBaseLoader(
    "https://news.naver.com/"
)

docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [ ]:
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()

retriever_tool = create_retriever_tool(
    retriever, "naver_news_search", "네이버 뉴스정보가 저장된 벡터 DB, 당일 뉴스 검색 가능"
)

arxiv_wrapper = ArxivAPIWrapper(
    top_k_results=1, doc_content_chars_max=200, load_all_available_meta=False,)

arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
tools = [wiki, retriever_tool, arxiv]

agent = create_openai_tools_agent(llm=openai, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
result = agent_executor.invoke({'input' : '트랜스포머 구조를 알려줘'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Transformer architecture'}`


Page: Transformer (deep learning architecture)
Summary: In deep learning, transformer is an architecture based on the multi-head attention mechanism, in which text is converted to numerical representa트랜스포머(Transformer) 구조는 딥러닝에서 사용되는 아키텍처로, 주로 자연어 처리(NLP) 분야에서 활용됩니다. 이 구조는 멀티 헤드 어텐션(multi-head attention) 메커니즘에 기반하여, 텍스트를 수치적 표현으로 변환하는 방식으로 작동합니다.

트랜스포머는 다음과 같은 주요 구성 요소로 이루어져 있습니다:

1. **인코더(Encoder)**: 입력 데이터를 처리하여 고차원 벡터로 변환합니다. 여러 개의 인코더 층으로 구성되어 있으며, 각 층은 멀티 헤드 어텐션과 피드포워드 신경망으로 이루어져 있습니다.

2. **디코더(Decoder)**: 인코더에서 생성된 벡터를 바탕으로 최종 출력을 생성합니다. 디코더 역시 여러 개의 층으로 구성되어 있으며, 인코더의 출력을 참고할 수 있는 어텐션 메커니즘을 포함합니다.

3. **어텐션 메커니즘(Attention Mechanism)**: 입력의 각 부분이 다른 부분에 얼마나 주의를 기울여야 하는지를 결정합니다. 멀티 헤드 어텐션은 여러 개의 어텐션 메커니즘을 병렬로 사용하여 다양한 표현을 학습할 수 있게 합니다.

4. **포지셔널 인코딩(Positional Encoding)**: 순서 정보가 없는 입력 데이터에 순서를 부여하기 위해 사용됩니다. 이는 모델이 단어의 위치 정보를 이해할 수 있도록 돕습니다.

트랜스포머 구조는 RNN이나 LSTM과 같은 